In [1]:
# Main import block
import random
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import numpy as np
import datetime
from datetime import date, datetime


import time
from tqdm import tqdm

# Upload to google sheets
import gspread
#import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

import signal
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')


In [2]:
##### Functions to maintain side manipulations #####


# Timeout class for reattempting connection
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


# Function to convert date from json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def date_format_reverse():
    date_now = date.today()
    date_string = date_now.strftime("%Y-%m-%d %H:%M:%S")
    date_raw = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").timestamp()
    date_raw = date_raw*1000
    return int(date_raw)


# Function for uploading dataframes into the google docs
def google_upload(df, sheet_name):
    # Params used to connect to google api
    scope = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        'macro-parser-lme-c2f2972b48fc.json', scope)  # security token
    gc = gspread.authorize(credentials)

    # Key params for connection to particular document
    spreadsheet_key = '1WhLiXRcdlkG7NCvHac9unC8ROt4lcbY7GxrOEdezZ9s'  # document id
    wks_name = sheet_name  # sheet name that we use
    d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials)
    # print(f'Uploading to {sheet_name} completed')


# Session creation via proxy
def get_session(url):

    # Free proxy function
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find('table').find_all('tr')  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all('td'))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == 'yes':  # taking only https
                    proxies[raw_list[i]
                            [3].text] = f'{raw_list[i][0].text}:{raw_list[i][1].text}'
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # create session
    session = requests.Session()

    # random proxy
    proxy = get_free_proxies()
    counter = 0

    while counter <= len(proxy):
        try:
            with time_limit(7):
                random_proxy = proxy.sample().values[0]
                session.proxies = {"https": random_proxy}
                response = session.get(url)
                break

        except OSError:
            pass

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1
            print(f'Attempt {counter+1}')

    return response


###### This is the main function block ######


################################################################
##############   LME ################
################################################################


def lme_db_addition():

    def get_day_info():

        # URL API for every metall, place into the var
        url_aluminium = 'https://www.lme.com/api/trading-data/day-delayed?datasourceId=1a0ef0b6-3ee6-4e44-a415-7a313d5bd771'
        url_copper = 'https://www.lme.com/api/trading-data/day-delayed?datasourceId=762a3883-b0e1-4c18-b34b-fe97a1f2d3a5'
        url_lead = 'https://www.lme.com/api/trading-data/day-delayed?datasourceId=bc443de6-0bdd-4464-8845-9504f528b0c6'
        url_nikel = 'https://www.lme.com/api/trading-data/day-delayed?datasourceId=acadf037-c13f-42f2-b42a-cac9a8179940'
        url_zink = 'https://www.lme.com/api/trading-data/day-delayed?datasourceId=c389e2b0-c4a3-46a0-96ca-69cacbe90ee4'

        # List for iterations
        req_list = {'aluminium': url_aluminium, 'copper': url_copper,
                    'lead': url_lead, 'nickel': url_nikel, 'zink': url_zink}

        # Empty dict for final row-stage
        day_dict = {'date': [], 'aluminium': [], 'copper': [],
                    'lead': [], 'nickel': [], 'zink': [], }

        # Getting json from api's requests and taking the info (in our case OFFER price for a date)
        for metal, url in req_list.items():
            req = requests.get(url).json()
            metal_dict = req['Rows'][0]
            day_dict[metal] = float(metal_dict['Values'][1])
            day_dict['date'] = metal_dict['BusinessDateTime']

        # Transform dict from previous stage into the row
        dict_ = dict(day_dict)  # Maybe can simplify this
        day_row = pd.DataFrame([dict_])
        day_row.date = pd.to_datetime(day_row.date)

        return day_row  # This is our row for implimentation into the main base

    # Opening main base, add a row, check for dupp, saving and closing
    lme_db = pd.read_excel(
        '../parser_beta/data/LME_db.xlsx', index_col=0)

    try:
        day_row = get_day_info()
        lme_db = pd.concat([lme_db, day_row], axis=0, ignore_index=True)
        lme_db.drop_duplicates(inplace=True)

    except IndexError:
        print('LME response is empty, please check the source.')

    with pd.ExcelWriter(
        "../parser_beta/data/LME_db.xlsx",
        date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        lme_db.to_excel(writer, sheet_name='LME_non_ferrous')
    # print('LME parsing is DONE!')

    google_upload(lme_db, 'LME_non_ferrous')

    return lme_db


################################################################
##############   KITCO ################
################################################################


def kitco_db():
    # In KITCO parsing we're taking slightly different aproach, we need to reupload
    # the table into the file because sometimes KITCO changing data backdating

    #!!!Need to work out the implementation of previous year data!!!!
    # year = int(date.today().year) - 2001

    url = 'https://www.kitco.com/gold.londonfix.html'
    # url_previous_year = f'https://www.kitco.com/londonfix/gold.londonfix{year}.html'

    # response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}) # ordinal responce
    response = get_session(url=url)  # response via proxy
    # responce_prev = requests.get(url_previous_year, headers={'User-Agent': 'Mozilla/5.0'})

    kitco_response = pd.read_html(response.text)
    kitco_df = pd.read_excel(
        '../parser_beta/data/kitko_db.xlsx', index_col=0)

    # Get the raw table and drop unnesesary rows
    kitco_day = kitco_response[1]
    kitco_day.drop([1, 4, 6], axis=1, inplace=True)
    kitco_day.columns = kitco_day.iloc[0]
    kitco_day.drop([0, 1, 2], axis=0, inplace=True)

    # Change tyoe of data within table
    kitco_day['Date'] = pd.to_datetime(kitco_day['Date'])
    kitco_day = kitco_day.replace({'-': np.nan})
    kitco_day = kitco_day.sort_values(by=['Date'])
    kitco_day = kitco_day.reset_index(drop=True)
    kitco_day[['Gold', 'Silver', 'Platinum', 'Palladium']] = kitco_day[[
        'Gold', 'Silver', 'Platinum', 'Palladium']].apply(pd.to_numeric)

    kitco_day.drop_duplicates(inplace=True)

    # And rewrite old table
    kitco_day.to_excel(
        '../parser_beta/data/kitko_db.xlsx', sheet_name='kitco_metall')
    # print('KITCO parsing is DONE!')

    google_upload(kitco_day, 'KITCO')

    return kitco_day


################################################################
##############   CB ################
################################################################


def cb_curr():
    day = date.today()
    today = day.strftime('%d/%m/%Y')

    dict_of_currencies = {
        'R01235': 'USD',
        'R01239': 'EUR',
        'R01010': 'Australian_Dollar',
        'R01375': 'China_Yuan',
        'R01035': 'British_Pound',
        'R01335': 'Kazakhstan_Tenge',
        'R01820': 'Japanese_Yen',
        'R01775': 'Swiss_Franc'
    }

    list_of_currencies = [x for x in dict_of_currencies.keys()]

    URL_list = []
    for currency in list_of_currencies:
        URL = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={today}&date_req2={today}&VAL_NM_RQ={currency}'
        URL_list.append(URL)

    currency_df = pd.read_excel(
        '../parser_beta/data/cb_curr.xlsx', index_col=0)

    # This problem occurs in the beginning of the year so I was forced to catch a ValueError

    try:
        for url_element in URL_list:
            response_df = pd.read_xml(url_element)
            response_df['Date'] = pd.to_datetime(
                response_df['Date'], dayfirst=True)
            response_df['Value'] = response_df['Value'].apply(
                lambda x: x.replace(',', '.'))
            response_df['Value'] = response_df['Value'].apply(pd.to_numeric)
            response_df = response_df.replace(dict_of_currencies)
            currency_df = pd.concat(
                [currency_df, response_df], axis=0, ignore_index=True)
            currency_df.drop_duplicates(inplace=True)
        # print('CentroBank_currency parsing is DONE!')

        with pd.ExcelWriter(
                "../parser_beta/data/cb_curr.xlsx") as writer:
            currency_df.to_excel(writer, sheet_name='curr')

        google_upload(currency_df, 'cb_curr')

    except ValueError:
        return 'Empty data set in CentroBank_currency. Should check the source.'

    return currency_df


def cb_metall():
    day = date.today()
    today = day.strftime('%d/%m/%Y')

    metall_dict = {
        1: 'gold',
        2: 'silver',
        3: 'platinum',
        4: 'palladium'
    }

    URL = f'http://www.cbr.ru/scripts/xml_metall.asp?date_req1={today}&date_req2={today}'

    metall_df = pd.read_excel(
        '../parser_beta/data/cb_metall.xlsx', index_col=0)

    # This problem occurs in the beginning of the year so I was forced to catch a ValueError
    try:
        response_df = pd.read_xml(URL)
        response_df.drop(columns='Buy', axis=1, inplace=True)
        response_df['Date'] = pd.to_datetime(
            response_df['Date'], dayfirst=True)
        response_df['Sell'] = response_df['Sell'].apply(
            lambda x: x.replace(',', '.'))  # changing for future retyping to numeric
        response_df['Sell'] = response_df['Sell'].apply(pd.to_numeric)
        response_df = response_df.replace(metall_dict)

        metall_df = pd.concat(
            [metall_df, response_df],
            axis=0,
            ignore_index=True)

        metall_df.drop_duplicates(inplace=True)
        # print('CentroBank_metalls parsing is DONE!')

        with pd.ExcelWriter(
            '../parser_beta/data/cb_metall.xlsx',
            date_format='YYYY-MM-DD',
                datetime_format='YYYY-MM-DD') as writer:
            metall_df.to_excel(writer, sheet_name='cb_metall')

        google_upload(metall_df, 'cb_metall')

    except ValueError:
        return 'Empty data set in CentroBank_metalls . Should check the source.'

    return metall_df


################################################################
##############   NBK ################
################################################################


def nbk_tenge():
    # Realy unrelieable source, mb it would be better off with using ms query inside the file
    year = date.today().year

    upper_bound = f'01.01.{year}'
    lower_bound = f'31.12.{year}'

    url = f'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut\
        /report?rates%5B%5D=5&beginDate={upper_bound}&endDate={lower_bound}'

    counter = 0

    while counter <= 6:
        try:
            with time_limit(15):
                page = requests.get(url=url)
                break

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1

    temp_df = pd.read_html(page.text)
    df = temp_df[0]
    df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])#, dayfirst=True)
    df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

    with pd.ExcelWriter(
            '../parser_beta/data/nbk_tenge.xlsx') as writer:
        df.to_excel(writer, sheet_name='tenge')

    # print('NBK_tenge parsing is DONE!')

    google_upload(df, 'nbk_tenge')

    return df

################################################################
##############   SHMET ################
################################################################

def shmet_parser():
    def shmet_day():
        day = date.today()
        today = date_format_reverse()
        # =1690186841910'
        url = f'https://en.shmet.com/api/rest/enweb/spot/getChartPrices?scId=811&startDate={day}&endDate={day}&_{today}'

        response = requests.get(url)
        if response.status_code == 200:  # Проверка успешности запроса
            data = response.json()  # Получение JSON-данных из ответа

        with open("./data/shmet.json", "w") as file:  # Открытие файла для записи
            json.dump(data, file)  # Запись JSON-данных в файл

        temp = pd.read_json('./data/shmet.json')['data']['spotPrices']
        data = pd.DataFrame(temp)

        data['dateTime'] = data['dateTime'].apply(date_format)
        data.rename({'dateTime': 'date'}, axis=1, inplace=True)
        data['date'] = pd.to_datetime(data['date']).dt.date
        data['date'] = pd.to_datetime(data['date'])

        return data

    df = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    df.drop_duplicates(inplace=True)

    try:
        day_row = shmet_day()
        shmet_db = pd.concat([day_row, df], axis=0)

        shmet_db = shmet_db.reset_index(drop=True)
        shmet_db.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
                "../parser_beta/data/shmet_historical.xlsx",
                date_format="YYYY-MM-DD",
                datetime_format="YYYY-MM-DD") as writer:
            shmet_db.to_excel(writer, sheet_name='SHMET')

    except (IndexError, KeyError) as e:
        print('Shmet day is empty, please check the source.')
        return None

    final = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    final.drop_duplicates(inplace=True)

    google_upload(final, 'SHMET')

    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final.to_excel(writer, sheet_name='SHMET')

    return final

################################################################
##############   WESTMETAL (dupplicate for LME) ################
################################################################


def westmetall():
    def aluminium():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Al_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        aluminium_date = pd.to_datetime(page_elements[0].text)
        aluminium_settlment = page_elements[1].text

        aluminium = pd.DataFrame(
            data={'date': aluminium_date, 'aluminium': aluminium_settlment}, index=[0])
        # aluminium['aluminium'] = pd.to_numeric(aluminium['aluminium'])

        return aluminium

    def copper():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Cu_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        copper_date = pd.to_datetime(page_elements[0].text)
        copper_settlment = page_elements[1].text

        copper = pd.DataFrame(
            data={'date': copper_date, 'copper': copper_settlment}, index=[0])
        # copper['copper'] = pd.to_numeric(copper['copper'])

        return copper

    def lead():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Pb_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        lead_date = pd.to_datetime(page_elements[0].text)
        lead_settlment = page_elements[1].text

        lead = pd.DataFrame(
            data={'date': lead_date, 'lead': lead_settlment}, index=[0])
        # lead['lead'] = pd.to_numeric(lead['lead'])

        return lead

    def nickel():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Ni_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        nickel_date = pd.to_datetime(page_elements[0].text)
        nickel_settlment = page_elements[1].text

        nickel = pd.DataFrame(
            data={'date': nickel_date, 'nickel': nickel_settlment}, index=[0])
        # nickel['nickel'] = pd.to_numeric(nickel['nickel'])

        return nickel

    def zink():
        url = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Zn_cash'
        response = requests.get(url)

        page = BeautifulSoup(response.text, 'html.parser')
        page_elements = page.find_all('td')

        zink_date = pd.to_datetime(page_elements[0].text)
        zink_settlment = page_elements[1].text

        zink = pd.DataFrame(
            data={'date': zink_date, 'zink': zink_settlment}, index=[0])
        # zink['zink'] = pd.to_numeric(zink['zink'])

        return zink

    al_data = aluminium()
    cu_data = copper()
    ld_data = lead()
    nk_data = nickel()
    zk_data = zink()

    result = pd.merge(pd.merge(pd.merge(pd.merge(al_data, cu_data, on='date', suffixes=['_al', '_cu']),
                                        ld_data, on='date'), nk_data, on='date'), zk_data, on='date')

    result['aluminium'] = result['aluminium'].str.replace(',', '')
    result['copper'] = result['copper'].str.replace(',', '')
    result['lead'] = result['lead'].str.replace(',', '')
    result['nickel'] = result['nickel'].str.replace(',', '')
    result['zink'] = result['zink'].str.replace(',', '')

    df = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)
    df = pd.concat([df, result], axis=0)
    df.drop_duplicates(inplace=True)
    df = df.reset_index(drop=True)

    with pd.ExcelWriter(
        "../parser_beta/data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        df.to_excel(writer, sheet_name='LME_westmetall')

    df = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)
    df.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
        "../parser_beta/data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        df.to_excel(writer, sheet_name='LME_westmetall')

    google_upload(df, 'LME_westmetall')

    return df


##### Run!!! #####


# if __name__ == '__main__':
#    functions = {
#        'LME': lme_db_addition, 'CB_CURR': cb_curr,
#        'CB_METAL': cb_metall, 'NBK': nbk_tenge, 'KITCO': kitco_db
#    }

#    for number, func in tqdm(functions.items()):
#        try:
#            func()

#        except ValueError:
#            print(f'Supposed problems on {number} side')


In [3]:
#lme_db_addition()

In [4]:
westmetall()

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730.0,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900.0,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480.0,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725.0,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045.0,3576.5
...,...,...,...,...,...,...
410,2023-08-21,2068.5,8239.5,2168.0,20050.0,2270.5
411,2023-08-22,2121.5,8342.0,2195.0,20215.0,2336.0
412,2023-08-23,2129.0,8377.5,2190.0,20525.0,2328.5
413,2023-08-24,2119.5,8358.5,2214.0,20400.0,2363.0


In [5]:
cb_curr()

'Empty data set in CentroBank_currency. Should check the source.'

In [6]:
cb_metall()

'Empty data set in CentroBank_metalls . Should check the source.'

In [7]:
nbk_tenge()

,date,Числовое значение,ДОЛЛАР США
0,2023-01-01,1,462.65
1,2023-01-02,1,462.65
2,2023-01-03,1,462.65
3,2023-01-04,1,462.65
4,2023-01-05,1,465.39
...,...,...,...
234,2023-08-23,1,453.13
235,2023-08-24,1,458.13
236,2023-08-25,1,462.04
237,2023-08-26,1,460.17


In [8]:
kitco_db()

,Date,Gold,Silver,Platinum,Palladium
0,2023-01-03,1843.25,24.295,1082.0,1795.0
1,2023-01-04,1857.30,24.290,1080.0,1736.0
2,2023-01-05,1834.00,23.410,1062.0,1783.0
3,2023-01-06,1852.20,23.455,1073.0,1784.0
4,2023-01-09,1878.85,23.850,1092.0,1793.0
...,...,...,...,...,...
160,2023-08-22,1892.75,23.390,928.0,1289.0
161,2023-08-23,1916.65,23.750,929.0,1282.0
162,2023-08-24,1917.05,24.185,933.0,1244.0
163,2023-08-25,1.36,NaN,949.0,1233.0


In [9]:
shmet_parser()

Shmet day is empty, please check the source.
